## 0. Setup

In [21]:
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Verificar que la API key está disponible
api_key = os.getenv("OPENAI_API_KEY")
print("API key cargada:", "Sí" if api_key else "No")



API key cargada: Sí


## 1. Extracting files

In [42]:
import os
from langchain_community.document_loaders import PyPDFLoader

# Intentar ambas rutas posibles
base_paths = ["data", "../data"]

pdf_files = ["guiaRouter.pdf", "recetaHotCakes.pdf"]
documents = []

for base in base_paths:
    if os.path.exists(base):
        for pdf in pdf_files:
            path = os.path.join(base, pdf)
            if os.path.exists(path):
                loader = PyPDFLoader(path)
                docs = loader.load()
                documents.extend(docs)
                print(f"{path} → {len(docs)} fragmentos")
            else:
                print(f"⚠️ No se encontró el archivo: {path}")
        break  # si encontró la carpeta base, ya no prueba la otra

print(f"Total cargado en todos los documentos: {len(documents)} fragmentos")


../data\guiaRouter.pdf → 2 fragmentos
../data\recetaHotCakes.pdf → 2 fragmentos
Total cargado en todos los documentos: 4 fragmentos


In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader

# Intentar ambas rutas posibles
base_paths = ["data", "../data"]

pdf_files = ["guiaRouter.pdf", "recetaHotCakes.pdf"]
documents = []

for base in base_paths:
    if os.path.exists(base):
        for pdf in pdf_files:
            path = os.path.join(base, pdf)
            if os.path.exists(path):
                loader = PyPDFLoader(path)
                docs = loader.load()
                documents.extend(docs)
                print(f"{path} → {len(docs)} fragmentos")
            else:
                print(f"⚠️ No se encontró el archivo: {path}")
        break  # si encontró la carpeta base, ya no prueba la otra

print(f"Total cargado en todos los documentos: {len(documents)} fragmentos")


../data\guiaRouter.pdf → 2 fragmentos
../data\recetaHotCakes.pdf → 2 fragmentos
Total cargado en todos los documentos: 4 fragmentos


## 2. Text Splitting into Chunks

In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configurar el splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # tamaño adecuado
    chunk_overlap=50,    # solapamiento adecuado
    separators=["\n\n", "\n", " ", ""]
)

# Aplicar el splitter a los documentos cargados
docs_chunked = text_splitter.split_documents(documents)

print(f"Fragmentos después del chunking: {len(docs_chunked)}")
print("Ejemplo de fragmento:\n")
print(docs_chunked[0].page_content[:300])  # mostrar solo 300 caracteres


Fragmentos después del chunking: 56
Ejemplo de fragmento:

Quick Setup - Wireless
Conectar el Hardware
Conecte un cable Ethernet del
módem al puerto WAN de 
su router.
Si su conexión a Internet es a través de un cable Ethernet de pared 
en lugar de a través de un DSL/ Cable/ Módem Satelital, conecte el 
cable Ehernet directamente al puerto WAN del router, l


## 3. Embedding

In [31]:
from langchain_openai import OpenAIEmbeddings

# Crear embeddings con OpenAI
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Convertir los fragmentos a vectores
embeddings = embedding_model.embed_documents([doc.page_content for doc in docs_chunked])

print(f"Se generaron {len(embeddings)} embeddings.")
print(f"Ejemplo de embedding (primer vector, 5 números):\n{embeddings[0][:5]}")


Se generaron 56 embeddings.
Ejemplo de embedding (primer vector, 5 números):
[0.026735464110970497, 0.007250760216265917, -0.01332201436161995, -0.025400519371032715, -0.01750057376921177]


## 4. Vector Stores

In [32]:
from langchain_community.vectorstores import Chroma

# Crear un almacén vectorial en memoria con ChromaDB
vectorstore = Chroma.from_documents(
    docs_chunked,
    embedding_model,
    persist_directory="../chroma_db"  # carpeta donde se guarda la base
)

print("Base vectorial creada y persistida en '../chroma_db'")


Base vectorial creada y persistida en '../chroma_db'


## 5. Retriving from the Persistant Vector Datastore

In [33]:
retriever = vectorstore.as_retriever()

query = "¿Cómo se prepara la mezcla para hot cakes?"
results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results, start=1):
    print(f"\nResultado {i}:")
    print(doc.page_content[:300])



Resultado 1:
Hot Cakes
Una vez integrados se agregan a la harina y SUAVEMENTE se mezclan juntos (no batir muy fuerte ni de
más porque luego se activa el gluten de la harina y quedarán duros).  Si queda muy espeso se puede
agregar leche hasta completar una taza, pero no más de eso.
La mezcla se tiene que dejar re

Resultado 2:
Hot Cakes
Tiempo de preparación: 10 minutos
Tiempo de cocimiento: 20 minutos
Tiempo total: 30 minutos
Porciones: 10 hot cakes medianos
Estos hot cakes son las noches de invierno en mi casa, o tal vez las tardes de verano cuando salíamos bien
refrescados de estar en la alberca toda la tarde.  Son mi 

Resultado 3:
http://www.tp-link.com/mx/support/faq y haga clic en Contactar a Soporte Técnico para asistencia.
1.
Configure el router usando un web browser2.
TerminarRegresar
¡Felicitaciones!
Los ajustes básicos de Internet e inalámbricas están terminados, por favor haga clicBotónde terminar 
y probar las conexi

Resultado 4:
http://www.tp-link.com/mx/support/faq y h

## 6. Retrivers in Langchain

In [34]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Crear el LLM (modelo de OpenAI para generar respuestas)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Crear un chain de RAG: primero recupera info, luego genera respuesta
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True  # opcional, para ver de dónde sacó la info
)

# Ejemplo de pregunta
query = "¿Qué ingredientes se necesitan para preparar hot cakes?"
response = qa_chain.invoke({"query": query})

print("Respuesta generada:\n", response["result"])

print("\nFuente(s):")
for doc in response["source_documents"]:
    print("-", doc.metadata)

query = "¿Cómo puedo reiniciar el router?"
response = qa_chain.invoke({"query": query})

print("Respuesta generada:\n", response["result"])
print("\nFuente(s):")
for doc in response["source_documents"]:
    print("-", doc.metadata)



Respuesta generada:
 Los ingredientes necesarios para preparar hot cakes son:
- 2 tazas de harina preparada para hot cakes.
- 2 huevos.
- 2 cucharadas de aceite.
- Entre ¾ y 1 taza de leche, dependiendo de la consistencia deseada.

Fuente(s):
- {'creationdate': "20250919054129+00'00'", 'title': 'Hot Cakes', 'source': '../data/recetaHotCakes.pdf', 'page_label': '2', 'page': 1, 'author': 'Cassarola Mia', 'creator': 'PyPDF', 'total_pages': 2, 'producer': 'mPDF 7.0.3', 'moddate': "20250919054129+00'00'"}
- {'total_pages': 2, 'author': 'Cassarola Mia', 'page': 0, 'source': '../data/recetaHotCakes.pdf', 'page_label': '1', 'moddate': "20250919054129+00'00'", 'creationdate': "20250919054129+00'00'", 'creator': 'PyPDF', 'title': 'Hot Cakes', 'producer': 'mPDF 7.0.3'}
- {'title': '7106506448_TL-WR840N(ES)_2.0_QIG_V1', 'producer': 'Adobe PDF library 10.01', 'page_label': '2', 'source': '../data/guiaRouter.pdf', 'moddate': '2016-06-12T10:31:56+08:00', 'total_pages': 2, 'page': 1, 'creator': 'Adobe

In [46]:
#FASE 4
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Crear el modelo y la cadena QA
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(model="gpt-4o-mini")  # o "gpt-3.5-turbo"
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 5 consultas de prueba
preguntas = [
    "¿Cómo se prepara la receta de hot cakes?",
    "¿Qué ingredientes principales llevan los hot cakes?",
    "Explica cómo se configura el router en palabras simples.",
    "¿Qué pasos básicos se recomiendan para instalar el router?",
    "Resume las instrucciones clave del manual del router."
]

for q in preguntas:
    print("❓", q)
    print("➡️", qa_chain.run(q))
    print("-" * 50)


❓ ¿Cómo se prepara la receta de hot cakes?


C:\Users\jose.gregorio\AppData\Local\Temp\ipykernel_20672\2270995755.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print("➡️", qa_chain.run(q))


➡️ Para preparar la receta de hot cakes, sigue estos pasos:

**Ingredientes:**
- 2 tazas de harina preparada para hot cakes.
- 2 huevos.
- 2 cucharadas de aceite.
- ¾ a 1 taza de leche (dependiendo de la consistencia deseada).

**Procedimiento:**
1. En un tazón hondo, coloca la harina y desbarata los posibles grumos con un tenedor. Es mejor si la cerniste o pasas por un colador.
2. En otro recipiente, mezcla los huevos, el aceite y ¾ de taza de leche. Bate muy bien con un batidor de globo.
3. Integra la mezcla de huevos a la harina y mezcla suavemente. Evita batir fuerte o en exceso para no activar el gluten de la harina.
4. Si la mezcla queda muy espesa, puedes agregar leche hasta completar una taza, pero no más de eso.
5. Deja reposar la mezcla al menos 10 minutos para que las proteínas del huevo y las partículas de harina absorban la leche.
6. Calienta un comal con cubierta antiadherente. Una vez caliente, pasa un poco de mantequilla con una servilleta y vierte la mezcla sin mover a